In [35]:
from manim import *
import numpy as np

In [46]:
%%manim -qm -v WARNING --disable_caching ICPScene
# --format=gif 

points = []
for x in np.linspace(0, 2, 4):
    points.append((x, 0))
    points.append((0, x))


class ICPScene(Scene):
    def construct(self):
        t = ValueTracker(0)
        t.add_updater(lambda x, dt: x.increment_value(dt))
        self.add(t)
        
        pc_map = VGroup()
        pc_reading = VGroup()
        for x, y in points:
            pc_map.add(Dot((x, y, 0), radius=0.1, color=YELLOW))
            pc_reading.add(Dot((x, y, 0), radius=0.1, color=RED))
        ax = Axes(
            x_range=[0, 5], 
            y_range=[0, 1, 10], 
            axis_config={'include_tip': False, 'include_numbers': False},
            x_axis_config={'include_ticks': False},
            y_axis_config={'include_ticks': False},
            x_length=6,
            y_length=2.5,
        )
        labels = ax.get_axis_labels(x_label="x", y_label="f(x)")
        f_x = lambda x: np.exp(-x**2)
        plot = ax.plot(f_x, x_range=(0, 5), use_smoothing=True, color=RED)
        
        graph = VGroup(ax, labels, plot)
        graph.shift(3 * LEFT)
        
        self.add(graph)
        
        # pc offset
        pc_reading.shift([-1, -1, 0])
        
        pc_map.shift(3 * RIGHT)
        pc_reading.shift(3 * RIGHT)
            
        lines = VGroup()
        for p1, p2 in zip(pc_map, pc_reading):
            lines.add(Line(p1.get_center(), p2.get_center()))
            
        def update_lines(lines, dt):
            for i in range(len(lines)):
                lines[i].become(Line(pc_map[i].get_center(), pc_reading[i].get_center()))
            
        def update_graph(plot, dt):
            plot.become(ax.plot(f_x, x_range=(0, t.get_value() + 1e-3), use_smoothing=True, color=RED))
            
        lines.add_updater(update_lines)
        plot.add_updater(update_graph)
            
        self.add(lines, pc_map, pc_reading)
        self.play(pc_reading.animate.shift([1, 1, 0]))
        self.wait(3)
        
        lines.clear_updaters()
        plot.clear_updaters()
        
        self.play(*[FadeOut(mob) for mob in self.mobjects])
        self.remove(*self.mobjects)
        
        title = MathTex(r'\sum_{n=1}^\infty \frac{1}{n^2} = \frac{\pi^2}{6}')
        self.add(title)
        self.play(FadeIn(title))
        self.wait(2)
        self.play(FadeOut(title))
        
        image = ImageMobject('img/norlab_logo_acronym_dark.png').scale(2)
        self.play(GrowFromCenter(image))
        self.wait(2)
        self.play(Wiggle(image, scale_value=1.5, rotation_angle=np.pi/5))
        self.play(FadeOut(image))
        
        equation = MathTex(
            r"e^x = x^0 + x^1 + \frac{1}{2} x^2 + \frac{1}{6} x^3 + \cdots + \frac{1}{n!} x^n + \cdots",
            substrings_to_isolate="x"
        )
        equation.set_color_by_tex("x", YELLOW)
        self.add(equation)
        self.wait(2)
